In [275]:
import os, sys
from pathlib import Path
import json
from collections import defaultdict

import webcolors
import colorsys

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode()


DATA_DIR = Path("../data")
movie_paths = [movie for movie in DATA_DIR.iterdir() if DATA_DIR.is_dir() and movie.is_dir()]

movie_json = {}
for movie in movie_paths:
    movie_name = movie.parts[-1]
    json_path = Path(movie, movie_name + ".json")
    if json_path.exists():
        movie_json[movie_name] = json.load(json_path.open("r"))

def grab_hue_buckets(movie_list):
    hue_buckets = defaultdict(list)
    for movie, frames in movie_json.items():
        if movie in movie_list:
            for frame in frames:
                for color in frame['palette']:
                    hex_code = list(color.keys())[0][1:]
                    r = int('0x' + hex_code[0:2], 0) / 255
                    g = int('0x' + hex_code[2:4], 0) / 255
                    b = int('0x' + hex_code[4:6], 0) / 255
                    hsv = colorsys.rgb_to_hsv(r, g, b)
                    hue_range = int((hsv[0] * 360) / 30)
                    hue_buckets[hue_range].append(frame)
    return hue_buckets

In [276]:
def rgb_color_format(hsv):
    #Brighten to beautify
    rgb = colorsys.hsv_to_rgb(hsv[0], hsv[1] + 0.5, hsv[2] + 0.3)
    return 'rgb(' + str(rgb[0] * 255) + ',' + str(rgb[1] * 255) + ',' + str(rgb[2] * 255) + ')'

def graph_color(hue_buckets): 
    scatters = []
    color_names = ['red', 'orangered', 'orange', 'gold', 'yellow', 'yellowgreen', 'green',
                   'darkturquoise', 'blue', 'blueviolet', 'violet', 'mediumvioletred']
    
    for index, hue_bucket in hue_buckets.items():
        color_name = color_names[index]
        color_hex = webcolors.name_to_hex(color_name)
        #print(color_name, color_hex)
        hue = []
        val = []
        color = []
        for frame in hue_bucket:
            hue.append(frame['averageColorHSV'][0])
            val.append(frame['averageColorHSV'][2])
            color.append(rgb_color_format(frame['averageColorHSV']))
        scatters.append(Scatter(x=hue,
                        y=val,
                        mode='markers',
                        name=color_name,
                        marker={'color': color}
                       ))
    
    layout = Layout(xaxis={'title': 'Hue'}, yaxis={'title': 'Value'})
    fig = Figure(data=scatters, layout=layout)
    iplot(fig)

In [277]:
#2010s
movie_list = ['budapest', 'cabin', 'otherguys']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [278]:
#2000s
movie_list = ['darkknight', 'killbill1', 'killbill2', 'mulholland', 'pianist', 'sincity', 'watchmen']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [279]:
#1990s
movie_list = ['eyeswidshut', 'fightclub', 'groundhog', 'magnolia', 'shawshank']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [280]:
#1980s
movie_list = ['et']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [281]:
#1970s
movie_list = ['jaws']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [282]:
#1960s
movie_list = ['graduate']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)

In [283]:
#1950s
movie_list = ['rearwindow', 'vertigo']
hue_buckets = grab_hue_buckets(movie_list)
graph_color(hue_buckets)